# 参考
https://github.com/ganguli-lab/Synaptic-Flow/blob/master/Pruners/pruners.py
generatemaskpy文件主体就是这里来的


# 环境 参数

In [1]:
import os 
import copy 
import torch
import argparse
import numpy as np
import torch.nn as nn
from timm.models import create_model
from datasets import build_dataset
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset, Dataset,TensorDataset
from torchvision.datasets.folder import ImageFolder

import models

from my_pruning_utils import vitheadPruner
#import pruning_utils
# 替换模型atten为sp-atten
from layers import Attention_Headmask,Attention_sp
from vision_transformer import Attention

#手动定义参数类，专门存储训练的参数，
class cfg(object):
    pass
args = cfg()#实例化

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#strip/gmp/amp_50_small.sh

args.sparsity=0.4
args.atten_density=0.5
args.mlp_density=0.5
args.heads=12
args.pretrained='https://dl.fbaipublicfiles.com/deit/deit_base_patch16_224-b5f2ef4d.pth'
args.save_file='deit_base_patch16_224_head0605.pt'
args.data=None
args.batch_size=128
args.type='headcut_grow'
args.model='deit_base_patch16_224'
args.input_size=224
args.drop=0.0
args.drop_path=0.1

lxytimm-init.py-timm.__version__ 0.6.13


/home/nucopilot/lxy/vtp/Baseline_pruning/my_pruning_utils.py:40: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if " " is not module.proj.bias:


# 函数

In [2]:
def check_atten(binary_tensor, dim=64):
    heads_num = binary_tensor.shape[0] // dim
    output_list = []
    for i in range(heads_num):
        activated = binary_tensor[i*dim:i*dim+dim].mean().byte().float()
        output_list.append(activated.item())
    return output_list

def prune_loop(model, loss, pruner, dataloader, device, sparsity, scope, epochs, train_mode=False):

    # Set model to train or eval mode
    model.train()
    if not train_mode:
        model.eval()

    # Prune model
    for epoch in range(epochs):
        pruner.score(model, loss, dataloader, device)
        sparse = sparsity**((epoch + 1) / epochs) # represent density actually.
        pruner.mask(sparse, scope)

def prune_head_linear(model):# 替换模型中的atten子模型--深度优先搜索（DFS）的遍历方式
    for name, module in reversed(model._modules.items()):
        if len(list(module.children())) > 0:
            model._modules[name] = prune_head_linear(model=module)
            #print("!!!n",name)
        if isinstance(module, Attention):
            layer_new = Attention_sp( dim= 768, num_heads=12, qkv_bias=True, qk_scale=None, attn_drop=0., proj_drop=0.)
            model._modules[name] = layer_new
            #print("!!!!!trans",name)
    return model

In [3]:
def prune_head_linearreversed(model):# 替换模型中的atten子模型
    for name, module in model._modules.items():
        if len(list(module.children())) > 0:
            model._modules[name] = prune_head_linearreversed(model=module)
        if isinstance(module, Attention):
            layer_new = Attention_sp( dim= 768, num_heads=12, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0.)
            model._modules[name] = layer_new   
    return model

In [4]:
def prune_head_linear_named_modules(model):# 替换模型中的atten子模型
    for name, module in model.named_modules():
        if len(list(module.children())) > 0:
            model._modules[name] = prune_head_linear_named_modules(model=module)
        if isinstance(module, Attention):
            layer_new = Attention_sp( dim= 768, num_heads=12, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0.)
            model._modules[name] = layer_new   

    return model

# main

In [5]:
if args.type == 'headcut_grow':

    print('#########################################')
    print('########## headcut_grow ####################')
    print('#########################################')

    number_examples = 128
    data = torch.ones(number_examples, 3, args.input_size, args.input_size)
    target = torch.ones(number_examples)
    data_set = torch.utils.data.TensorDataset(data, target)
    loader = torch.utils.data.DataLoader(data_set, 
        batch_size=1, shuffle=False, 
        num_workers=2, pin_memory=True)


#########################################
########## headcut_grow ####################
#########################################


In [6]:

if args.type == 'headcut_grow':
    print(f"Creating model: {args.model}")
    model = create_model(
        args.model,
        pretrained=False,
        num_classes=1000,
        drop_rate=args.drop,
        drop_path_rate=args.drop_path,
        drop_block_rate=None,
    )
    


Creating model: deit_base_patch16_224


In [7]:
model.blocks[0].attn

Attention(
  (qkv): Linear(in_features=768, out_features=2304, bias=True)
  (attn_drop): Dropout(p=0.0, inplace=False)
  (proj): Linear(in_features=768, out_features=768, bias=True)
  (proj_drop): Dropout(p=0.0, inplace=False)
)

In [8]:
# for name, module in model._modules.items():
#     print(name)
'''
patch_embed
pos_drop
gumbel
blocks
norm
pre_logits
head

'''


'\npatch_embed\npos_drop\ngumbel\nblocks\nnorm\npre_logits\nhead\n\n'

## load pretrained weight

In [9]:
if args.type == 'headcut_grow':
    print('loading pretrained weight')
    checkpoint = torch.hub.load_state_dict_from_url(
        args.pretrained, map_location='cpu', check_hash=True)
    model.load_state_dict(checkpoint['model'],strict=False)#改了一下，不然提示Missing key(s) in state_dict: "gumbel.weight", "gumbel.bias". 

    save_state_dict = copy.deepcopy(model.state_dict())


loading pretrained weight


## 替换模型的conv和linear？？
看layers.py ，新建的层里面有self.register_buffer('weight_mask', torch.ones(self.weight.shape))

In [10]:

if args.type == 'headcut_grow':    
    prune_head_linear(model)#moedl._modules[name] = layer_new  新的conv fc


In [11]:
# list(model.blocks[0].attn.named_children())
# model.blocks[0].attn.proj.weight.data
# list(model.blocks[0].attn.proj.named_parameters(recurse=False))
# list(model.blocks[0].attn.named_buffers())

## 把之前deepcopy的参数重新加载回去

In [12]:

if args.type == 'headcut_grow':
    for key in save_state_dict.keys():
        if not key in model.state_dict().keys():
            print('can not load key = {}'.format(key))

    model.load_state_dict(save_state_dict, strict=False)
    model.cuda()


In [13]:
save_state_dict.keys()#原版的与训练好的model
#odict_keys(['cls_token', 'pos_embed', 'patch_embed.proj.weight', 'patch_embed.proj.bias', 'gumbel.weight', 'gumbel.bias', 
# 'blocks.0.norm1.weight', 'blocks.0.norm1.bias', 'blocks.0.attn.qkv.weight', 'blocks.0.attn.qkv.bias', 'blocks.0.attn.proj.weight', 'blocks.0.attn.proj.bias', 'blocks.0.norm2.weight', 'blocks.0.norm2.bias', 'blocks.0.mlp.fc1.weight', 'blocks.0.mlp.fc1.bias', 'blocks.0.mlp.fc2.weight', 'blocks.0.mlp.fc2.bias', 
# 'blocks.1.norm1.weight', 'blocks.1.norm1.bias', 'blocks.1.attn.qkv.weight', 'blocks.1.attn.qkv.bias', 'blocks.1.attn.proj.weight', 'blocks.1.attn.proj.bias', 'blocks.1.norm2.weight', 'blocks.1.norm2.bias', 'blocks.1.mlp.fc1.weight', 'blocks.1.mlp.fc1.bias', 'blocks.1.mlp.fc2.weight', 'blocks.1.mlp.fc2.bias', 'blocks.2.norm1.weight', 'blocks.2.norm1.bias', 'blocks.2.attn.qkv.weight', 'blocks.2.attn.qkv.bias', 'blocks.2.attn.proj.weight', 'blocks.2.attn.proj.bias', 'blocks.2.norm2.weight', 'blocks.2.norm2.bias', 'blocks.2.mlp.fc1.weight', 'blocks.2.mlp.fc1.bias', 'blocks.2.mlp.fc2.weight', 'blocks.2.mlp.fc2.bias', 'blocks.3.norm1.weight', 'blocks.3.norm1.bias', 'blocks.3.attn.qkv.weight', 'blocks.3.attn.qkv.bias', 'blocks.3.attn.proj.weight', 'blocks.3.attn.proj.bias', 'blocks.3.norm2.weight', 'blocks.3.norm2.bias', 'blocks.3.mlp.fc1.weight', 'blocks.3.mlp.fc1.bias', 'blocks.3.mlp.fc2.weight', 'blocks.3.mlp.fc2.bias', 'blocks.4.norm1.weight', 'blocks.4.norm1.bias', 'blocks.4.attn.qkv.weight', 'blocks.4.attn.qkv.bias', 'blocks.4.attn.proj.weight', 'blocks.4.attn.proj.bias', 'blocks.4.norm2.weight', 'blocks.4.norm2.bias', 'blocks.4.mlp.fc1.weight', 'blocks.4.mlp.fc1.bias', 'blocks.4.mlp.fc2.weight', 'blocks.4.mlp.fc2.bias', 'blocks.5.norm1.weight', 'blocks.5.norm1.bias', 'blocks.5.attn.qkv.weight', 'blocks.5.attn.qkv.bias', 'blocks.5.attn.proj.weight', 'blocks.5.attn.proj.bias', 'blocks.5.norm2.weight', 'blocks.5.norm2.bias', 'blocks.5.mlp.fc1.weight', 'blocks.5.mlp.fc1.bias', 'blocks.5.mlp.fc2.weight', 'blocks.5.mlp.fc2.bias', 'blocks.6.norm1.weight', 'blocks.6.norm1.bias', 'blocks.6.attn.qkv.weight', 'blocks.6.attn.qkv.bias', 'blocks.6.attn.proj.weight', 'blocks.6.attn.proj.bias', 'blocks.6.norm2.weight', 'blocks.6.norm2.bias', 'blocks.6.mlp.fc1.weight', 'blocks.6.mlp.fc1.bias', 'blocks.6.mlp.fc2.weight', 'blocks.6.mlp.fc2.bias', 'blocks.7.norm1.weight', 'blocks.7.norm1.bias', 'blocks.7.attn.qkv.weight', 'blocks.7.attn.qkv.bias', 'blocks.7.attn.proj.weight', 'blocks.7.attn.proj.bias', 'blocks.7.norm2.weight', 'blocks.7.norm2.bias', 'blocks.7.mlp.fc1.weight', 'blocks.7.mlp.fc1.bias', 'blocks.7.mlp.fc2.weight', 'blocks.7.mlp.fc2.bias', 'blocks.8.norm1.weight', 'blocks.8.norm1.bias', 'blocks.8.attn.qkv.weight', 'blocks.8.attn.qkv.bias', 'blocks.8.attn.proj.weight', 'blocks.8.attn.proj.bias', 'blocks.8.norm2.weight', 'blocks.8.norm2.bias', 'blocks.8.mlp.fc1.weight', 'blocks.8.mlp.fc1.bias', 'blocks.8.mlp.fc2.weight', 'blocks.8.mlp.fc2.bias', 'blocks.9.norm1.weight', 'blocks.9.norm1.bias', 'blocks.9.attn.qkv.weight', 'blocks.9.attn.qkv.bias', 'blocks.9.attn.proj.weight', 'blocks.9.attn.proj.bias', 'blocks.9.norm2.weight', 'blocks.9.norm2.bias', 'blocks.9.mlp.fc1.weight', 'blocks.9.mlp.fc1.bias', 'blocks.9.mlp.fc2.weight', 'blocks.9.mlp.fc2.bias', 'blocks.10.norm1.weight', 'blocks.10.norm1.bias', 'blocks.10.attn.qkv.weight', 'blocks.10.attn.qkv.bias', 'blocks.10.attn.proj.weight', 'blocks.10.attn.proj.bias', 'blocks.10.norm2.weight', 'blocks.10.norm2.bias', 'blocks.10.mlp.fc1.weight', 'blocks.10.mlp.fc1.bias', 'blocks.10.mlp.fc2.weight', 'blocks.10.mlp.fc2.bias', 'blocks.11.norm1.weight', 'blocks.11.norm1.bias', 'blocks.11.attn.qkv.weight', 'blocks.11.attn.qkv.bias', 'blocks.11.attn.proj.weight', 'blocks.11.attn.proj.bias', 'blocks.11.norm2.weight', 'blocks.11.norm2.bias', 'blocks.11.mlp.fc1.weight', 'blocks.11.mlp.fc1.bias', 'blocks.11.mlp.fc2.weight', 'blocks.11.mlp.fc2.bias', 'norm.weight', 'norm.bias', 'head.weight', 'head.bias'])
model.state_dict().keys()#这个里面多了headmask的buffers
#odict_keys(['cls_token', 'pos_embed', 'patch_embed.proj.weight', 'patch_embed.proj.bias', 'gumbel.weight', 'gumbel.bias', 
# 'blocks.0.norm1.weight', 'blocks.0.norm1.bias', 'blocks.0.attn.head_mask', 'blocks.0.attn.qkv.weight', 'blocks.0.attn.qkv.bias', 'blocks.0.attn.proj.weight', 'blocks.0.attn.proj.bias', 'blocks.0.norm2.weight', 'blocks.0.norm2.bias', 'blocks.0.mlp.fc1.weight', 'blocks.0.mlp.fc1.bias', 'blocks.0.mlp.fc2.weight', 'blocks.0.mlp.fc2.bias', 
# 'blocks.1.norm1.weight', 'blocks.1.norm1.bias', 'blocks.1.attn.head_mask', 'blocks.1.attn.qkv.weight', 'blocks.1.attn.qkv.bias', 'blocks.1.attn.proj.weight', 'blocks.1.attn.proj.bias', 'blocks.1.norm2.weight', 'blocks.1.norm2.bias', 'blocks.1.mlp.fc1.weight', 'blocks.1.mlp.fc1.bias', 'blocks.1.mlp.fc2.weight', 'blocks.1.mlp.fc2.bias', 'blocks.2.norm1.weight', 'blocks.2.norm1.bias', 'blocks.2.attn.head_mask', 'blocks.2.attn.qkv.weight', 'blocks.2.attn.qkv.bias', 'blocks.2.attn.proj.weight', 'blocks.2.attn.proj.bias', 'blocks.2.norm2.weight', 'blocks.2.norm2.bias', 'blocks.2.mlp.fc1.weight', 'blocks.2.mlp.fc1.bias', 'blocks.2.mlp.fc2.weight', 'blocks.2.mlp.fc2.bias', 'blocks.3.norm1.weight', 'blocks.3.norm1.bias', 'blocks.3.attn.head_mask', 'blocks.3.attn.qkv.weight', 'blocks.3.attn.qkv.bias', 'blocks.3.attn.proj.weight', 'blocks.3.attn.proj.bias', 'blocks.3.norm2.weight', 'blocks.3.norm2.bias', 'blocks.3.mlp.fc1.weight', 'blocks.3.mlp.fc1.bias', 'blocks.3.mlp.fc2.weight', 'blocks.3.mlp.fc2.bias', 'blocks.4.norm1.weight', 'blocks.4.norm1.bias', 'blocks.4.attn.head_mask', 'blocks.4.attn.qkv.weight', 'blocks.4.attn.qkv.bias', 'blocks.4.attn.proj.weight', 'blocks.4.attn.proj.bias', 'blocks.4.norm2.weight', 'blocks.4.norm2.bias', 'blocks.4.mlp.fc1.weight', 'blocks.4.mlp.fc1.bias', 'blocks.4.mlp.fc2.weight', 'blocks.4.mlp.fc2.bias', 'blocks.5.norm1.weight', 'blocks.5.norm1.bias', 'blocks.5.attn.head_mask', 'blocks.5.attn.qkv.weight', 'blocks.5.attn.qkv.bias', 'blocks.5.attn.proj.weight', 'blocks.5.attn.proj.bias', 'blocks.5.norm2.weight', 'blocks.5.norm2.bias', 'blocks.5.mlp.fc1.weight', 'blocks.5.mlp.fc1.bias', 'blocks.5.mlp.fc2.weight', 'blocks.5.mlp.fc2.bias', 'blocks.6.norm1.weight', 'blocks.6.norm1.bias', 'blocks.6.attn.head_mask', 'blocks.6.attn.qkv.weight', 'blocks.6.attn.qkv.bias', 'blocks.6.attn.proj.weight', 'blocks.6.attn.proj.bias', 'blocks.6.norm2.weight', 'blocks.6.norm2.bias', 'blocks.6.mlp.fc1.weight', 'blocks.6.mlp.fc1.bias', 'blocks.6.mlp.fc2.weight', 'blocks.6.mlp.fc2.bias', 'blocks.7.norm1.weight', 'blocks.7.norm1.bias', 'blocks.7.attn.head_mask', 'blocks.7.attn.qkv.weight', 'blocks.7.attn.qkv.bias', 'blocks.7.attn.proj.weight', 'blocks.7.attn.proj.bias', 'blocks.7.norm2.weight', 'blocks.7.norm2.bias', 'blocks.7.mlp.fc1.weight', 'blocks.7.mlp.fc1.bias', 'blocks.7.mlp.fc2.weight', 'blocks.7.mlp.fc2.bias', 'blocks.8.norm1.weight', 'blocks.8.norm1.bias', 'blocks.8.attn.head_mask', 'blocks.8.attn.qkv.weight', 'blocks.8.attn.qkv.bias', 'blocks.8.attn.proj.weight', 'blocks.8.attn.proj.bias', 'blocks.8.norm2.weight', 'blocks.8.norm2.bias', 'blocks.8.mlp.fc1.weight', 'blocks.8.mlp.fc1.bias', 'blocks.8.mlp.fc2.weight', 'blocks.8.mlp.fc2.bias', 'blocks.9.norm1.weight', 'blocks.9.norm1.bias', 'blocks.9.attn.head_mask', 'blocks.9.attn.qkv.weight', 'blocks.9.attn.qkv.bias', 'blocks.9.attn.proj.weight', 'blocks.9.attn.proj.bias', 'blocks.9.norm2.weight', 'blocks.9.norm2.bias', 'blocks.9.mlp.fc1.weight', 'blocks.9.mlp.fc1.bias', 'blocks.9.mlp.fc2.weight', 'blocks.9.mlp.fc2.bias', 'blocks.10.norm1.weight', 'blocks.10.norm1.bias', 'blocks.10.attn.head_mask', 'blocks.10.attn.qkv.weight', 'blocks.10.attn.qkv.bias', 'blocks.10.attn.proj.weight', 'blocks.10.attn.proj.bias', 'blocks.10.norm2.weight', 'blocks.10.norm2.bias', 'blocks.10.mlp.fc1.weight', 'blocks.10.mlp.fc1.bias', 'blocks.10.mlp.fc2.weight', 'blocks.10.mlp.fc2.bias', 'blocks.11.norm1.weight', 'blocks.11.norm1.bias', 'blocks.11.attn.head_mask', 'blocks.11.attn.qkv.weight', 'blocks.11.attn.qkv.bias', 'blocks.11.attn.proj.weight', 'blocks.11.attn.proj.bias', 'blocks.11.norm2.weight', 'blocks.11.norm2.bias', 'blocks.11.mlp.fc1.weight', 'blocks.11.mlp.fc1.bias', 'blocks.11.mlp.fc2.weight', 'blocks.11.mlp.fc2.bias', 'norm.weight', 'norm.bias', 'head.weight', 'head.bias'])

odict_keys(['cls_token', 'pos_embed', 'patch_embed.proj.weight', 'patch_embed.proj.bias', 'gumbel.weight', 'gumbel.bias', 'blocks.0.norm1.weight', 'blocks.0.norm1.bias', 'blocks.0.attn.qkv_mask', 'blocks.0.attn.proj_mask', 'blocks.0.attn.qkv.weight', 'blocks.0.attn.qkv.bias', 'blocks.0.attn.proj.weight', 'blocks.0.attn.proj.bias', 'blocks.0.norm2.weight', 'blocks.0.norm2.bias', 'blocks.0.mlp.fc1.weight', 'blocks.0.mlp.fc1.bias', 'blocks.0.mlp.fc2.weight', 'blocks.0.mlp.fc2.bias', 'blocks.1.norm1.weight', 'blocks.1.norm1.bias', 'blocks.1.attn.qkv_mask', 'blocks.1.attn.proj_mask', 'blocks.1.attn.qkv.weight', 'blocks.1.attn.qkv.bias', 'blocks.1.attn.proj.weight', 'blocks.1.attn.proj.bias', 'blocks.1.norm2.weight', 'blocks.1.norm2.bias', 'blocks.1.mlp.fc1.weight', 'blocks.1.mlp.fc1.bias', 'blocks.1.mlp.fc2.weight', 'blocks.1.mlp.fc2.bias', 'blocks.2.norm1.weight', 'blocks.2.norm1.bias', 'blocks.2.attn.qkv_mask', 'blocks.2.attn.proj_mask', 'blocks.2.attn.qkv.weight', 'blocks.2.attn.qkv.bias

## pruner包装

masked_parameters()
Returns an iterator over models prunable parameters, yielding both the mask and parameter tensors.

Mag基于Pruner（见pruning_utils.py）  ,主要重写的score

类似的
https://github.com/facebookresearch/open_lth/blob/main/pruning/test/test_pruned_model.py

https://github.com/facebookresearch/open_lth

王欢-https://github.com/MingSun-Tse

https://github.com/MingSun-Tse/Regularization-Pruning

https://github.com/MingSun-Tse/Why-the-State-of-Pruning-so-Confusing


196的图剪枝，也实现了一些基础的pruner

In [14]:

if args.type == 'headcut_grow':
    
    pruner = vitheadPruner(model)


Attention_sp blocks.0.attn
proj_mask proj
Attention_sp blocks.1.attn
proj_mask proj
Attention_sp blocks.2.attn
proj_mask proj
Attention_sp blocks.3.attn
proj_mask proj
Attention_sp blocks.4.attn
proj_mask proj
Attention_sp blocks.5.attn
proj_mask proj
Attention_sp blocks.6.attn
proj_mask proj
Attention_sp blocks.7.attn
proj_mask proj
Attention_sp blocks.8.attn
proj_mask proj
Attention_sp blocks.9.attn
proj_mask proj
Attention_sp blocks.10.attn
proj_mask proj
Attention_sp blocks.11.attn
proj_mask proj


In [15]:
pruner.name_proj_masks

[('blocks.0.attn.proj_mask',
  tensor([[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]], device='cuda:0')),
 ('blocks.1.attn.proj_mask',
  tensor([[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]], device='cuda:0')),
 ('blocks.2.attn.proj_mask',
  tensor([[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]], device='cuda:0')),
 ('blocks.3.attn.proj_mask',
  tensor([[1., 1., 1., 

In [16]:
# if 1:
#     if 1:
#         if 1:
#             remain_indicator = [0,1,1,1,0,0,0,0,1,1,1,1]
#             muti_head_dim = 64
#             heads = 12
#             atten_dim = muti_head_dim*heads

#             atten_mask = {}



#             for i,(nmask,mask ,param)  in enumerate(pruner.masked_parameters):
#                 attn_mask_name = str(i)+ nmask + '.qkv.weight_mask'
#                 print("param",param.size())
#                 atten_mask[attn_mask_name] = torch.zeros_like(param)
#                 for j,index_inside_layer in enumerate(remain_indicator):
#                     if index_inside_layer:

#                         q_start = j * muti_head_dim
#                         k_start = q_start + atten_dim
#                         v_start = k_start + atten_dim 
#                         atten_mask[attn_mask_name][q_start: q_start + muti_head_dim, :] = 1.
#                         atten_mask[attn_mask_name][k_start: k_start + muti_head_dim, :] = 1.
#                         atten_mask[attn_mask_name][v_start: v_start + muti_head_dim, :] = 1.
#                 zero = torch.tensor([0.]).to(mask.device)
#                 one = torch.tensor([1.]).to(mask.device)
#                 attenmask = torch.tensor(atten_mask[attn_mask_name],dtype=torch.bool)
#                 mask.copy_(torch.where(attenmask, one, zero))
#             print("!!!!!!!!!!!!",atten_mask.keys())

In [17]:
#check the dim  --mask
# for i,(nmask,mask ,param)  in enumerate(pruner.masked_parameters):
#     print(i,nmask)
#     a0 =  torch.sum(mask,dim=0)#torch.Size([768])
#     a1 =  torch.sum(mask,dim=1)#torch.Size([2304])    
#     print(a0)
#     print(a1)

#check the dim  -- atten_mask
# a0 =  torch.sum(atten_mask['0head_mask.qkv.weight_mask'],dim=0)#torch.Size([768])
# a1 =  torch.sum(atten_mask['0head_mask.qkv.weight_mask'],dim=1)#torch.Size([2304])
# a0


# no

# atten_mask['0head_mask.qkv.weight_mask'].data[0]

# for nmask,mask, param  in pruner.masked_parameters:
#     #print(mask.size(), param.size())#torch.Size([768, 3, 16, 16]) torch.Size([768, 3, 16, 16])
#     print(mask.size())
#     print(nmask,param.size())

In [18]:
pruner.scores#{}

{}

## loop 看源码 拆开长这个样子

In [19]:

# if args.type == 'headcut_grow':
#     prune_loop(model, None, pruner, loader, torch.device('cuda:0'), args.sparsity, scope='global', epochs=1, train_mode=True)
#     print('Density = {}'.format(args.sparsity))


In [20]:
epoch = 0
epochs = 1
scope='headcut'
model.train()


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (gumbel): Linear(in_features=768, out_features=1, bias=True)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention_sp(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): Laye

In [21]:

pruner.head_proj_mag_score( 64)



In [22]:
pruner.scores
''' 
{'0proj_mask': tensor([ 61.0781,  76.3469,  41.0036,  79.8307,  42.6134,  55.4206,  69.6618,
          60.3581,  83.4860,  47.6483, 136.6528,  71.9564]),
 '1proj_mask': tensor([96.6375, 42.8315, 86.7008, 79.3395, 86.0602, 78.2636, 84.0477, 79.4786,
         80.2388, 91.3839, 82.6904, 58.4772]),
 '2proj_mask': tensor([75.6504, 81.5267, 72.1166, 78.0375, 81.4394, 68.6793, 84.6924, 92.0338,
         95.4294, 83.0186, 76.2529, 90.4616]),
 '3proj_mask': tensor([88.1573, 84.3942, 92.6462, 71.4680, 70.0141, 73.5328, 93.8461, 80.1168,
         72.1433, 86.1504, 70.6177, 92.0897]),
 '4proj_mask': tensor([ 79.0642,  95.9358,  76.5184,  73.9169,  89.5061,  80.2325,  78.1621,
          98.1823, 100.7334,  98.8319,  81.2744,  74.7204]),
 '5proj_mask': tensor([ 81.6566,  93.7900,  96.3685,  94.5702,  95.5379, 103.4052,  89.9030,
         105.3707,  91.6535,  84.9583,  87.4861,  80.4489]),
 '6proj_mask': tensor([ 92.0345,  74.2973,  96.3436,  93.1568, 106.8964, 139.0841,  94.1444,
          98.7247,  90.1604, 101.9800,  90.1994,  95.7601]),
 '7proj_mask': tensor([102.9318, 111.9201, 107.8920,  96.1839, 100.6158, 136.0764, 117.4120,
         122.7663, 111.6184,  96.6562,  98.6268,  98.2645]),
 '8proj_mask': tensor([123.0539, 117.8744, 126.7822, 106.3182, 126.0455, 121.0203, 111.1697,
         107.2123, 122.7421, 130.7222, 113.1400, 114.9969]),
 '9proj_mask': tensor([116.7330, 128.2923, 128.9301, 116.4871, 119.2041, 119.0377, 119.9595,
         128.7403, 124.1525, 128.3461, 122.2350, 119.1208]),
 '10proj_mask': tensor([135.2132, 141.7011, 143.2963, 137.6145, 145.9363, 138.3276, 138.1556,
         142.5483, 147.0185, 137.2347, 131.7823, 136.7445]),
 '11proj_mask': tensor([183.1971, 176.6178, 164.9427, 182.5552, 197.1266, 170.5204, 160.6123,
         170.3534, 180.4831, 184.5145, 181.9834, 185.8626])}

'''

" \n{'0proj_mask': tensor([ 61.0781,  76.3469,  41.0036,  79.8307,  42.6134,  55.4206,  69.6618,\n          60.3581,  83.4860,  47.6483, 136.6528,  71.9564]),\n '1proj_mask': tensor([96.6375, 42.8315, 86.7008, 79.3395, 86.0602, 78.2636, 84.0477, 79.4786,\n         80.2388, 91.3839, 82.6904, 58.4772]),\n '2proj_mask': tensor([75.6504, 81.5267, 72.1166, 78.0375, 81.4394, 68.6793, 84.6924, 92.0338,\n         95.4294, 83.0186, 76.2529, 90.4616]),\n '3proj_mask': tensor([88.1573, 84.3942, 92.6462, 71.4680, 70.0141, 73.5328, 93.8461, 80.1168,\n         72.1433, 86.1504, 70.6177, 92.0897]),\n '4proj_mask': tensor([ 79.0642,  95.9358,  76.5184,  73.9169,  89.5061,  80.2325,  78.1621,\n          98.1823, 100.7334,  98.8319,  81.2744,  74.7204]),\n '5proj_mask': tensor([ 81.6566,  93.7900,  96.3685,  94.5702,  95.5379, 103.4052,  89.9030,\n         105.3707,  91.6535,  84.9583,  87.4861,  80.4489]),\n '6proj_mask': tensor([ 92.0345,  74.2973,  96.3436,  93.1568, 106.8964, 139.0841,  94.1444,\n  

 有了socre了  12bloscks里面12个头 一共144个
 接下来根据分数产生list

In [23]:
pruner.strategy

{}

In [24]:
pruner.score_to_list()

In [25]:
pruner.strategy

{'0proj_mask': [0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1],
 '1proj_mask': [0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1],
 '2proj_mask': [0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1],
 '3proj_mask': [0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1],
 '4proj_mask': [0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1],
 '5proj_mask': [0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1],
 '6proj_mask': [0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1],
 '7proj_mask': [0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1],
 '8proj_mask': [0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1],
 '9proj_mask': [0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1],
 '10proj_mask': [0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1],
 '11proj_mask': [0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1]}

 有了策略
 根据策略剪枝

In [26]:
#pruner.name_proj_masks_with_module[0]#'blocks.0.attn.qkv_mask'  blocks.0.attn.proj_mask  proj_mask

In [27]:
pruner.mask_by_list(64)

look
mask 0proj_mask torch.Size([768, 768])
mask 1proj_mask torch.Size([768, 768])
mask 2proj_mask torch.Size([768, 768])
mask 3proj_mask torch.Size([768, 768])
mask 4proj_mask torch.Size([768, 768])
mask 5proj_mask torch.Size([768, 768])
mask 6proj_mask torch.Size([768, 768])
mask 7proj_mask torch.Size([768, 768])
mask 8proj_mask torch.Size([768, 768])
mask 9proj_mask torch.Size([768, 768])
mask 10proj_mask torch.Size([768, 768])
mask 11proj_mask torch.Size([768, 768])


/home/nucopilot/lxy/vtp/Baseline_pruning/my_pruning_utils.py:40: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if " " is not module.proj.bias:
/home/nucopilot/lxy/vtp/Baseline_pruning/my_pruning_utils.py:40: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if " " is not module.proj.bias:


AttributeError: 'Tensor' object has no attribute 'keys'

In [ ]:
# for k in pruner.scores.keys():
#     print(k)
# sparse = args.sparsity**((epoch + 1) / epochs) # represent density actually. --0.4
# pruner.mask(sparse, scope)

# print('Density = {}'.format(args.sparsity))

## 后处理

In [ ]:

# if args.type == 'headcut_grow':
#     current_mask = extract_mask(model.state_dict())
#     check_sparsity_dict(current_mask)# 打印一下实际剩余参数占原本参数的rate 有没有到之前的 sparsity


## 保存

In [ ]:

# if args.type == 'headcut_grow':
#     torch.save(current_mask, args.save_file)